In [1]:
import ipyleaflet
import ipywidgets
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import datetime

In [2]:
path_inventario = '../LHC_Hidroweb/invent2.csv'

In [3]:
df = pd.read_csv(path_inventario, engine='python', sep='\t', delimiter=';', parse_dates=['UltimaAtualizacao'])

In [4]:
df.columns

Index(['RegistroID', 'Importado', 'Temporario', 'Removido',
       'ImportadoRepetido', 'BaciaCodigo', 'SubBaciaCodigo', 'RioCodigo',
       'EstadoCodigo', 'MunicipioCodigo', 'ResponsavelCodigo',
       'ResponsavelUnidade', 'ResponsavelJurisdicao', 'OperadoraCodigo',
       'OperadoraUnidade', 'OperadoraSubUnidade', 'TipoEstacao', 'Codigo',
       'Nome', 'CodigoAdicional', 'Latitude', 'Longitude', 'Altitude',
       'AreaDrenagem', 'TipoEstacaoEscala', 'TipoEstacaoRegistradorNivel',
       'TipoEstacaoDescLiquida', 'TipoEstacaoSedimentos',
       'TipoEstacaoQualAgua', 'TipoEstacaoPluviometro',
       'TipoEstacaoRegistradorChuva', 'TipoEstacaoTanqueEvapo',
       'TipoEstacaoClimatologica', 'TipoEstacaoPiezometria',
       'TipoEstacaoTelemetrica', 'PeriodoEscalaInicio', 'PeriodoEscalaFim',
       'PeriodoRegistradorNivelInicio', 'PeriodoRegistradorNivelFim',
       'PeriodoDescLiquidaInicio', 'PeriodoDescLiquidaFim',
       'PeriodoSedimentosInicio', 'PeriodoSedimentosFim',
      

In [5]:
df[['Latitude', 'Longitude']] = df[['Latitude', 'Longitude']].apply(lambda x: x.str.replace(',','.'))
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')

In [6]:
df[['Latitude','Longitude']].head()

,Latitude,Longitude
0,-13.6500,-73.0833
1,-13.0167,-73.9833
2,-13.6500,-73.3667
3,-13.4333,-73.8333
4,-13.6500,-73.9500


In [7]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude,df.Latitude), crs='epsg:4674')

In [62]:
m01 = ipyleaflet.Map(zoom=2)

# Layers
try:
    path_shapefile = r'G:\Meu Drive\USP-SHS\Outros\Shapefile\Jaguaribe\Jaguaribe.shp'
    shape = gpd.read_file(path_shapefile)
    geo_data = ipyleaflet.GeoData(geo_dataframe=shape, name='Bacias',style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2})
    m01.add_layer(geo_data)
except:
    pass

## All points for HEATMAP
heatmap_all = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],radius=30, name='All point Heatmap')
m01.add_layer(heatmap_all)

radius_slider = ipywidgets.IntSlider(description='Radius', min=1, max=50, value=15)
ipywidgets.jslink((radius_slider,'value'),(heatmap_all,'radius'))
widget_control01 = ipyleaflet.WidgetControl(widget=radius_slider, position='bottomright')
m01.add_control(widget_control01)

## Last collection day
def date_location(_):
    heatmap_byLast.locations = [tuple(s) for s in df.loc[df['UltimaAtualizacao'] > date_slider.value, ['Latitude','Longitude']].to_numpy()]

heatmap_byLast = ipyleaflet.Heatmap(locations=[tuple(r) for r in df[['Latitude', 'Longitude']].to_numpy()],radius=30, name='By Date')
m01.add_layer(heatmap_byLast)

ipywidgets.jslink((radius_slider,'value'),(heatmap_byLast,'radius'))


date_slider = ipywidgets.SelectionSlider(options=pd.date_range(start='2000-01-01',end='2020-01-01', freq='M').to_numpy())
date_slider.observe(date_location,names='value')
widget_control02 = ipyleaflet.WidgetControl(widget=date_slider, position='topright')
m01.add_control(widget_control02)


teste = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.NASAGIBS.ModisTerraTrueColorCR, "2020-01-30")


# Controles
## Layer
control_layer = ipyleaflet.LayersControl(position='topright')
m01.add_control(control_layer)

## Split Layer
# split_layer = ipyleaflet.SplitMapControl(left_layer=teste, right_layer=heatmap_byLast)
# m01.add_control(split_layer)

## Fullscreen
control_fullscreen = ipyleaflet.FullScreenControl()
m01.add_control(control_fullscreen)

## Desenho
control_desenho = ipyleaflet.DrawControl()
feature_collection = {'type': 'FeatureCollection',
                      'features': []}
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""   
    feature_collection['features'].append(geo_json)

control_desenho.on_draw(handle_draw)
m01.add_control(control_desenho)

## Escala
control_escala = ipyleaflet.ScaleControl(position='bottomleft')
m01.add_control(control_escala)

## Select
# box_layout = ipywidgets.Layout(display='flex',
#                     flex_flow='row',
#                     align_items='stretch',
#                     width='50%')

control_selectDownload = ipywidgets.Dropdown(options=['Watershed', 'All', 'byDate'])
control_buttom = ipywidgets.Button(description='Download')
vbox = ipywidgets.HBox([control_selectDownload, control_buttom])
widget_control03 = ipyleaflet.WidgetControl(widget=vbox, position='bottomright')
m01.add_control(widget_control03)

def download_buttom(_):
    try:
        last_draw = feature_collection['features'][-1]['geometry']
        last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
        print(last_polygon)
    except:
        print('Nenhum polígono desenhado')
        
    if control_selectDownload.value == 'All':
        sub = gdf.loc[(gdf['geometry'].within(last_polygon)),['Nome', 'Codigo']]
        sub.to_csv(r'C:\Users\Usuario\Desktop\teste.csv')
        
    elif control_selectDownload.value == 'byDate':
        sub = gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,['Nome', 'Codigo']]
        sub.to_csv(r'C:\Users\Usuario\Desktop\teste1.csv')
        
    elif control_selectDownload.value == 'Watershed':
        for i in shape['geometry']:
            sub = gdf.loc[gdf['geometry'].within(i), ['Nome', 'Codigo']]
            sub.to_csv(r'C:\Users\Usuario\Desktop\teste1.csv')

            
        
control_buttom.on_click(download_buttom)
m01

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [81]:
for index, row in shape.iterrows():
#     print(j['DESCRICA1'],j['geometry'])

SyntaxError: unexpected EOF while parsing (<ipython-input-81-50d678dc885f>, line 2)

In [ ]:
for k in range(len(teste_shp['DESCRICA1'])):
    print(teste_shp['DESCRICA1'][k])
    print(gdf.loc[gdf['geometry'].within(teste_shp['geometry'][k]), ['Nome', 'Codigo']])
    sub = gdf.loc[gdf['geometry'].within(teste_shp['geometry'][k]), ['Nome', 'Codigo']]
    print(sub['Codigo'].tolist())
#     sub.to_csv(r'G:\Meu Drive\USP-SHS\Outros\Shapefile\{}.csv'.format(teste_shp['DESCRICA1'][k]))

In [38]:
print(control_selectDownload.value)

All


In [10]:
try:
    last_draw = feature_collection['features'][-1]['geometry']
    last_polygon = Polygon([(i[0], i[1]) for i in last_draw['coordinates'][0]])
    print(last_polygon)
except:
    print('Nenhum polígono desenhado')

Nenhum polígono desenhado


In [11]:
gdf.loc[(gdf['geometry'].within(last_polygon)),['Nome', 'Codigo']]

NameError: name 'last_polygon' is not defined

In [11]:
gdf.loc[(gdf['geometry'].within(last_polygon)) & (gdf['UltimaAtualizacao']>date_slider.value) ,['Nome', 'Codigo']]

NameError: name 'last_polygon' is not defined

In [12]:
# path_shapefile = r'G:\Meu Drive\USP-SHS\Documentos_Word_Excel_PowerPoint\Shapefiles\35MUE250GC_SIR.shp'
# path_shapefile = r'G:\Meu Drive\USP-SHS\Documentos_Word_Excel_PowerPoint\Shapefiles\bioma.shp'
path_shapefile = r'G:\Meu Drive\USP-SHS\Outros\Shapefile\Jaguaribe\Jaguaribe.shp'

In [13]:
teste_shp = gpd.read_file(path_shapefile)
# city = teste_shp.loc[teste_shp['NM_MUNICIP']=='DRACENA']
# city
teste_shp

,fid,CODIGOBA0,DESCRICA1,IDENTIFI2,AREA3,HECTARES4,REGIONAL5,geometry
0,1.0,1,Alto Jaguaribe,11,25242.6908681,2524269.08681,Iguatu,"POLYGON ((-40.06977 -5.42811, -40.06888 -5.428..."
1,2.0,3,Baixo Jaguaribe,9,7112.41591989,711241.591989,Limoeiro do Norte,"POLYGON ((-37.78887 -4.39875, -37.78726 -4.399..."
2,3.0,7,Medio Jaguaribe,10,10352.3747726,1035237.47726,Limoeiro do Norte,"POLYGON ((-38.14588 -5.35235, -38.14305 -5.355..."
3,4.0,9,Salgado,12,12718.6950865,1271869.50865,Crato,"POLYGON ((-38.78977 -6.29794, -38.78946 -6.302..."
4,5.0,10,BanabuiÃº,8,19647.3615063,1964736.15063,Quixeramobim,"MULTIPOLYGON (((-38.70358 -4.94514, -38.70359 ..."


In [14]:
for i in teste_shp['geometry']:
#     print(i)
    print(gdf.loc[gdf['geometry'].within(i), ['Nome', 'Codigo']])
    sub = gdf.loc[gdf['geometry'].within(i), ['Nome', 'Codigo']]
#     sub.to_csv(r'G:\Meu Drive\USP-SHS\Outros\Shapefile\Jaguaribe\')

                      Nome  Codigo
85                  TRUSSU  639050
87     FAZENDA MORADA NOVA  639075
179                TANQUES  639047
181                  CAIXA  639046
182               GUARIBAS  639048
...                    ...     ...
33342        VILA ANTONICO  639068
33343             ARNEIROZ  639051
33345    CACHOEIRA DE FORA  640031
33359             ARNEIROZ  640042
33364         CAMPOS SALES  740006

[263 rows x 2 columns]
                     Nome    Codigo
84                 BIXOPA    438074
86      LIMOEIRO DO NORTE    538009
180         SÍTIO MALHADA    538059
1859      SÍTIO ÁGUA FRIA    438098
1869      SÍTIO ÁGUA FRIA  36630000
...                   ...       ...
33195   SÍTIO CAMPO LIMPO    437039
33196        LAGOA GRANDE    438090
33197               PEIXE    438091
33330  TABULEIRO DO NORTE    538069
33333  OLHO DAGUA DA BICA    538001

[83 rows x 2 columns]
                   Nome    Codigo
7117         ALTO SANTO  36385000
7228       ALTO RECREIO    538004

In [15]:
teste_shp['DESCRICA1']

0     Alto Jaguaribe
1    Baixo Jaguaribe
2    Medio Jaguaribe
3            Salgado
4          BanabuiÃº
Name: DESCRICA1, dtype: object

In [16]:
# for i in range(len(teste_shp['DESCRICA1'])):
#     print(teste_shp['DESCRICA1'][i])
#     print(teste_shp['geometry'][i])
# #     for j in 

Alto Jaguaribe
POLYGON ((-40.06976702632288 -5.428105373884663, -40.06887984006107 -5.428441778110782, -40.06262910552783 -5.427534544136922, -40.06162753871377 -5.43095434546375, -40.06226428621898 -5.431900490107325, -40.06449597618118 -5.435216527947103, -40.06219920256258 -5.436021610500798, -40.059555141036 -5.437449954696552, -40.05938682815103 -5.437166721795649, -40.05915542957683 -5.436894042923963, -40.0588802976509 -5.436665774785661, -40.05856979223348 -5.436488853750155, -40.05823334897776 -5.436368654958057, -40.05788118843357 -5.436308830026788, -40.05752401260995 -5.436311197236242, -40.05717267274959 -5.436375684222916, -40.05687617852892 -5.437041450964071, -40.05901395099131 -5.442428953261317, -40.05668297811189 -5.444841568665209, -40.05829813854145 -5.452569869165885, -40.05697381242254 -5.454636714571173, -40.05781235786606 -5.46337568040552, -40.05101533321794 -5.46460277649448, -40.05091819447568 -5.465134775031451, -40.05044414418098 -5.466685213114712, -40.04

In [18]:
for k in range(len(teste_shp['DESCRICA1'])):
    print(teste_shp['DESCRICA1'][k])
    print(gdf.loc[gdf['geometry'].within(teste_shp['geometry'][k]), ['Nome', 'Codigo']])
    sub = gdf.loc[gdf['geometry'].within(teste_shp['geometry'][k]), ['Nome', 'Codigo']]
    print(sub['Codigo'].tolist())
#     sub.to_csv(r'G:\Meu Drive\USP-SHS\Outros\Shapefile\{}.csv'.format(teste_shp['DESCRICA1'][k]))

Alto Jaguaribe
                      Nome  Codigo
85                  TRUSSU  639050
87     FAZENDA MORADA NOVA  639075
179                TANQUES  639047
181                  CAIXA  639046
182               GUARIBAS  639048
...                    ...     ...
33342        VILA ANTONICO  639068
33343             ARNEIROZ  639051
33345    CACHOEIRA DE FORA  640031
33359             ARNEIROZ  640042
33364         CAMPOS SALES  740006

[263 rows x 2 columns]
[639050, 639075, 639047, 639046, 639048, 639049, 639021, 740010, 639035, 640004, 640020, 639041, 36180000, 36100000, 639015, 740042, 740007, 740011, 640032, 640033, 640034, 640011, 640046, 640006, 640008, 639005, 639022, 639040, 36180001, 36190501, 640007, 639004, 639020, 36160001, 639010, 36080003, 36081000, 36080002, 540001, 540017, 640010, 640013, 640017, 640022, 640023, 640024, 640026, 640025, 639019, 639031, 36185000, 36190500, 640001, 36140000, 36150000, 740012, 540099, 540100, 540101, 540102, 540104, 540105, 540106, 540103, 6390

In [19]:
[539037, 36470000, 539078, 36460001, 538087, 36580000, 36586000, 538090, 539083, 539053, 36520000, 36515000, 538034, 539000, 439000, 36521000, 36520200, 36525000, 539049, 36595000, 36598000, 438030, 36590000, 36585000, 439077, 439076, 439064, 439063, 439062, 439082, 539070, 539002, 539073, 539086, 539044, 539047, 539048, 539045, 36470001, 36460000, 539022, 539034, 539038, 440011, 440025, 538000, 538014, 538019, 36584000, 439028, 539024, 539025, 539050, 539010, 539017, 36520001, 439004, 539019, 539080, 539081, 538092, 538093, 538094, 538095, 438045, 538011, 538013, 538021, 539006, 539018, 438025, 538037, 36550000, 538035, 36527000, 539042, 539009, 439039, 36500000, 439012, 539026, 539035, 539007, 539016, 439011, 439026, 438046, 438048, 36536000, 538006, 538012, 539061, 539021, 539033, 539062, 539092, 539039, 539003, 539093, 539096, 539005, 439065, 538060, 540039, 538031, 538032, 440029, 538103, 538075, 438105, 438072, 539077, 539064, 539069, 539066, 539065, 539068, 539072, 539012, 539067, 539095, 539097, 539088, 539084, 539090, 539091, 439083, 539074, 439074, 439001, 439020, 439022, 439067, 538063, 539063, 538064, 438117, 538018, 439080, 538083, 538051, 438024, 439073, 439043, 439044, 539056, 539057, 539055, 539094, 439045, 539089, 539029, 439060, 439061, 439059, 538047, 539054, 538025, 538097, 439002, 539027, 539028, 539031, 539082, 539079, 539043, 36517500, 538067, 36471000, 36594000, 539030, 539001, 539004, 539008, 539014, 36462000, 36512000, 36400000, 36461000, 36459000, 36599000, 36516000, 36593000, 36592500, 36770000, 36518000, 36519000, 36545000, 36565000, 36510000, 36490000, 36517000, 538027, 36522000, 36534000, 36458000, 539060, 539085, 539051, 539059, 540043]

In [1]:
print('teste')

teste
